In [70]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import re
from itertools import zip_longest

In [43]:
df = pd.read_csv('./animals_info.csv',encoding='ISO-8859-1')
print(df.dtypes)

Name               object
Kingdom            object
Phylum             object
Subphylum          object
Class              object
Order              object
Suborder           object
Family             object
Genus              object
Species            object
Population size    object
Life span          object
Top speed          object
Weight             object
Height             object
Length             object
Attributes         object
Distribution       object
Habits             object
Diet               object
Mating_Habits      object
Population         object
dtype: object


In [ ]:
# data aim
# 1. clean data
# 2. seperate data into category (animal type, physical characteristics, habitat, diet, etc)

In [7]:
# things to achieve
# 1. Kingdom - some data have repeated specimen, 
# create two folder to leminate the different
kingdom_anomoloy = df[df["Kingdom"].duplicated(keep=False) == False]
df = df[df["Kingdom"].duplicated(keep=False)]
kingdom_anomoloy.to_csv('unique_values.csv', index=False)
df.to_csv('ori.csv', index=False)



In [ ]:
# 2. Phylum
# 3. Subphylum - state become short form, make new column for district
# 4. Class - change to number
# 5. order - dump or important?
# 6. suborder - kiv, maybe - rank their role, executive, administrator, lead, assistant
# 7. family - to common format
# 8. genus - to common format
# 9. species - to common format


In [100]:
df = pd.read_csv('./animals_info.csv',encoding='ISO-8859-1')

def change_type(numbers, types):
    temp = []
    for number, type1 in zip_longest(numbers, types, fillvalue='none'):
        convert = 1.0

        # if not number:
        #     number.append(0)
        #     continueS

        # if re.search(r',', number) and len(number[i]) > 1:
        # if re.search(r',', number):
        #     number = number.replace(',', '.')

        if re.search(r'b', type1, flags=re.IGNORECASE):
            if re.search(r'^(below|above)$', type1, flags=re.IGNORECASE):
                convert = 1000000000
        elif re.search(r'm', type1, flags=re.IGNORECASE):
            convert = 1000000 
        elif re.search(r'\d+[kt]', type1, flags=re.IGNORECASE):
            convert = 1000 
        
        for i in number:
            temp = convert * float(''.join(i.split(',')))
    
    return temp

def get_min_max(numbers, types):

    min_value = numbers[0] if len(numbers) > 1 else None
    max_value = numbers[1] if len(numbers) > 1 else numbers[0]
    
    for type in types:
        if re.search(r'^(below)$', type, flags=re.IGNORECASE):
            min_value = 0
            max_value = numbers[0]
        elif re.search(r'^(above)$', type, flags=re.IGNORECASE):
            min_value = numbers[0]
            max_value = 0
        
    return min_value, max_value

def transform_population(population):
    if not isinstance(population, str):  # Handle non-string values
        return pd.Series([0, 0, None, None])

    population = re.sub(r'[^\x00-\x7F]+', ' ', population)  # Remove non-ASCII characters
    numbers = re.findall(r'\d+(?:,\d+)*(?:\.\d+)?', population)
    type = re.findall(r'[a-zA-Z]+', population)

    if not numbers:  # If no numbers are found
        return pd.Series([0, 0, population]) 

    numbers = change_type(numbers, type)
    min_value, max_value = get_min_max(numbers, type)

    # min_value = re.sub(r',', '', min_value)
    # max_value = re.sub(r',', '', max_value)

    # min_value = str(min_value) if min_value is not None else '0'
    # max_value = str(max_value) if max_value is not None else '0'
    
    # min_value = min_value.fillna(0).astype(int)
    # max_value = max_value.fillna(0).astype(int)

    # return pd.Series([int(float(min_value)), int(float(max_value)), type, numbers])
    return pd.Series([min_value, max_value, type, numbers])

df[['population_min', 'population_max','population_others', 'other']] = df['Population size'].apply(transform_population)

ValueError: could not convert string to float: ''

In [12]:
print(df.dtypes)


population_min       object
population_max       object
population_others    object
Population size      object
dtype: object


In [96]:
df = df[['population_min', 'population_max','population_others','other', 'Population size']]

df.to_csv('ori.csv', index=False)

In [ ]:
# 11. life span - make min & max. Blank to 0
# 12. top speed - make min & max. Blank to 0
# 13. weight -  make min & max. Blank to 0
# 14. height -  make min & max. Blank to 0
# 15. length -  make min & max. Blank to 0
# 16. attributes - kiv, maybe - rank their role, executive, administrator, lead, assistant
# 17. distribution - to common format
# 18. habits - to common format
# 19. diet - to common format
# 20. mating habits - similarity (consistent letter case, only sdn bhd capital)
# 21. populationn

In [ ]:
# # 10. population size - make min & max. Blank to 0
# def change_mil_thous(min, max, strings_at_back):
#     convert = 1000

#     if re.search(r'm', strings_at_back, flags=re.IGNORECASE):
#         convert = 1000000

#     if re.search(r',', strings_at_back):
#         min = float(min.replace(',', '.'))
#         max = float(max.replace(',', '.'))

#     return min*convert, max*convert

# def transform_population(population):
#     strings_at_back = []

#     if isinstance(population, str):
#         population = re.sub(r'[^\x00-\x7F]+', ' ', population)   #original    

#         numbers = re.findall(r'\d+(?:,\d+)*(?:\.\d+)?', population) #1,234, 5678, 90.12
        
#         if not numbers:
#             return pd.Series([None, None, population])
        
#         min_value = numbers[0] if len(numbers) > 1 else None
#         max_value = numbers[1] if len(numbers) > 1 else numbers[0]

#         parts = population.split()
#         if parts and not parts[-1].isdigit():
#             strings_at_back.append(parts[-1])
        
#         # look for k or m
#         if strings_at_back:
#             min, max = change_mil_thous(min_value, max_value, strings_at_back)

#         min = re.sub(r',', '', min)
#         max = re.sub(r',', '', max)

#         return pd.Series([min, max, None])
    
#     return pd.Series([None, None, None])


# df[['population_min', 'population_max','population_others']] = df['Population size'].apply(transform_population)